In [2]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

import tensorflow as tf
from tensorflow import keras, lite

from keras.models import load_model
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.callbacks import TensorBoard

from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

import datetime
from operator import index

In [3]:
mp_hands = mp.solutions.hands  # Holistic model
mp_drawing = mp.solutions.drawing_utils  # Drawing utilities


def mediapipe_detection(image, model):
    # COLOR CONVERSION BGR 2 RGB
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)  # COLOR COVERSION RGB 2 BGR
    return image, results


def draw_styled_landmarks(image, results):
    # Draw left hand connections
    if not results.multi_hand_landmarks:
        return
    for hand_landmarks in results.multi_hand_landmarks:
        mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS,
                                  mp_drawing.DrawingSpec(
                                      color=(111, 22, 76), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(
                                      color=(121, 44, 250), thickness=2, circle_radius=2)
                                  )


def draw_landmarks(image, results):
    if not results.multi_hand_landmarks:
        return
    for hand_landmarks in results.multi_hand_landmarks:
        # Draw left hand connections
        mp_drawing.draw_landmarks(
            image, hand_landmarks, mp_hands.HAND_CONNECTIONS)


def extract_keypoints(results):
    lh = np.array([[res.landmark[point].x, res.landmark[point].y, res.landmark[point].z]
                  for point in mp_hands.HandLandmark for res in results.multi_hand_landmarks], dtype=float).flatten() if results.multi_hand_landmarks else np.zeros(21*3*2)
    if lh.shape[0] == 63:
        rh = np.array([[res.landmark[point].x, res.landmark[point].y, res.landmark[point].z]
                      for point in mp_hands.HandLandmark for res in results.multi_hand_landmarks], dtype=float).flatten() if results.multi_hand_landmarks else np.zeros(21*3)
        return np.concatenate([lh, rh])
    return np.concatenate([lh])


colors = [
    (255, 0, 0),   # Red
    (0, 255, 0),   # Green
    (0, 0, 255),   # Blue
    (255, 255, 0),  # Yellow
    (255, 0, 255),  # Magenta
    (0, 255, 255),  # Cyan
    (128, 128, 128)  # Gray
]


def prob_viz(res, actions, input_frame, colors):
    output_frame = cv2.resize(input_frame, (1000, 800))
    cv2.rectangle(output_frame, (0, 0), (800, 60), (255, 255, 255), -1)
    cv2.putText(output_frame, 'Gesture Detection', (10, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 0), 2, cv2.LINE_AA)

    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (50, 100 + num * 50),
                      (50 + int(prob * 700), 130 + num * 50), colors[num], -1)
        cv2.putText(output_frame, f'{actions[num]}: {prob:.2f}', (
            60, 125 + num * 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

    return output_frame

In [4]:
# Actions that we try to detect
actions = np.array(['thumbsup', 'thumbsdown', 'peace', 'palm', 'fist'])

log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [5]:
model = load_model("action.h5")

In [6]:
# final code

sequence = []
sentence = []
predictions = []
threshold = 0.9
# timestamp = datetime.datetime.now().strftime("%Y%m%d")
# counter = 1

try:

    cap = cv2.VideoCapture(0)
    with mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5) as hands:
        start_time = time.time()
        while cap.isOpened():

            # Read feed
            ret, frame = cap.read()

            # Make detections
            image, results = mediapipe_detection(frame, hands)

            # Draw landmarks
            draw_styled_landmarks(image, results)

            # Prediction logic
            keypoints = extract_keypoints(results)
            sequence.append(keypoints)
            sequence = sequence[-30:]

            if len(sequence) == 30:
                res = model.predict(np.expand_dims(sequence, axis=0))[0]
                predictions.append(np.argmax(res))

                # Viz logic
                if np.unique(predictions[-10:])[0] == np.argmax(res):
                    if res[np.argmax(res)] > threshold:

                        word = ""
                        if len(sentence) > 0:
                            if actions[np.argmax(res)] != sentence[-1]:
                                sentence.append(actions[np.argmax(res)])
                                word = actions[np.argmax(res)]
                        else:
                            sentence.append(actions[np.argmax(res)])
                            word = actions[np.argmax(res)]

                        if word == "thumbsup":
                            # Save the photo with a unique filename
                            delay = 5000
                            # filename = "photo_{}.png".format((timestamp)_(counter))
                            filename = "photo_{}.png".format(counter)
                            cv2.imwrite(filename, frame)
                            print("Photo saved as", filename)
                            # Break out of the loop after taking one photo
                            delay = 10000

                        if word == "palm":
                            # Save the photo with a unique filename
                            delay = 5000
                            cv2.imshow(filename, frame)
                            # Break out of the loop after taking one photo
                            delay = 10000
                            cv2.destroyWindow()

                        if word == "thumbsdown":
                            # Delete the photo if the gesture is thumbs up
                            if os.path.exists(filename):
                                os.remove(filename)
                                print("Photo deleted:", filename)
                            else:
                                print("File does not exist:", filename)

                if len(sentence) > 1:
                    sentence = sentence[-1:]

                # Viz probabilities
                image = prob_viz(res, actions, image, colors)

            # Show to screen
            cv2.imshow('OpenCV Feed', image)

            # Break gracefully
            if cv2.waitKey(10) & 0xFF == 27:
                break

        cap.release()
        cv2.destroyAllWindows()

finally:
    cap.release()
    cv2.destroyAllWindows()

1/1 [==============================] - 0s 32ms/step


NameError: name 'filename' is not defined